In [154]:
import os
import webbrowser
from datetime import datetime
from openpyxl import Workbook
import requests
import html5lib
import numpy as np
import pandas as pd
import unicodedata
import re
from bs4 import BeautifulSoup

import helper


def parse_and_trim(content, content_type):
    if content_type == "HTML":
        soup = BeautifulSoup(content, 'html.parser')
    else:
        soup = BeautifulSoup(content, 'html.parser')
    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass
    for linebreak in soup.find_all('br'):
        linebreak.extract()
    return soup

In [155]:
headers = {
    'User-Agent': 'APOLLO INVESTMENT CORPORATION',
    'User-Agent': 'MIDCAP FINANCIAL INVESTMENT CORPORATION'
}
filing_links = pd.read_excel('../MFIC__sec_filing_links.xlsx')
date_columns = ['Filing date', 'Reporting date']
for col in date_columns:
    filing_links[col] = pd.to_datetime(filing_links[col], format='%Y-%m-%d')
for col in date_columns:
    filing_links[col] = filing_links[col].dt.strftime("%B %d, %Y")
filing_links.head(5)

,Form type,Form description,Filing date,Reporting date,url
0,10-Q,Quarterly report [Sections 13 or 15(d)],"November 07, 2023","September 30, 2023",https://www.sec.gov/Archives/edgar/data/127875...
1,10-Q,Quarterly report [Sections 13 or 15(d)],"August 02, 2023","June 30, 2023",https://www.sec.gov/Archives/edgar/data/127875...
2,10-Q,Quarterly report [Sections 13 or 15(d)],"May 02, 2023","March 31, 2023",https://www.sec.gov/Archives/edgar/data/127875...
3,10-KT,Transition reports [Rule 13a-10 or 15d-10],"February 21, 2023","December 31, 2022",https://www.sec.gov/Archives/edgar/data/127875...
4,10-Q,Quarterly report [Sections 13 or 15(d)],"November 03, 2022","September 30, 2022",https://www.sec.gov/Archives/edgar/data/127875...


In [156]:
for index, (url, reporting_date) in enumerate(zip(filing_links['url'], filing_links['Reporting date'])):
    pass

In [157]:
response = helper.get_response(
    url=filing_links['url'].iloc[1], headers=headers)
filing_links['url'].iloc[1]
content = helper.get_content(response)

In [158]:
content = helper.get_content(response)

In [159]:
consolidated_schedule_regex = re.compile(
    r'(?i)^\s*.*\s*SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')

for tag in content.find_all(text=consolidated_schedule_regex):
    print("Tag:", tag)
    print("Find_next:", tag.find_next())
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    print("Next_date:", re.search(date_regex_pattern1, tag.find_next().text))
    print("next:", tag.next)
    print("Next next: ", tag.find_next().next.next.next.text)
    print("next sib: ", tag.find_next_sibling())
    print("\n")

Tag: Schedule of Investments 
Find_next: <td><p><span>5</span></p></td>
Next_date: None
next: June 30, 2023
Next next:  5
next sib:  None


Tag: Schedule of Investments 
Find_next: <td><p><span>35</span></p></td>
Next_date: None
next: December 31, 2022
Next next:  35
next sib:  None


Tag: SCHEDULE OF INVESTMENTS (Unaudited)
Find_next: <p><span>June 30, 2023</span></p>
Next_date: <re.Match object; span=(0, 13), match='June 30, 2023'>
next: 

Next next:  

next sib:  None


Tag: SCHEDULE OF INVESTMENTS (Unaudited)
Find_next: <p><span>June 30, 2023</span></p>
Next_date: <re.Match object; span=(0, 13), match='June 30, 2023'>
next: 

Next next:  

next sib:  None


Tag: SCHEDULE OF INVESTMENTS (Unaudited)
Find_next: <p><span>June 30, 2023</span></p>
Next_date: <re.Match object; span=(0, 13), match='June 30, 2023'>
next: 

Next next:  

next sib:  None


Tag: SCHEDULE OF INVESTMENTS (Unaudited)
Find_next: <p><span>June 30, 2023</span></p>
Next_date: <re.Match object; span=(0, 13), match='Ju

/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_11149/3212395028.py:4: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for tag in content.find_all(text=consolidated_schedule_regex):


In [160]:
def extract_tables(soup_content, qtr_date):
    master_table = None
    consolidated_schedule_regex = re.compile(
        r'(?i)^\s*.*\s*SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):
        date_str = re.search(date_regex_pattern1, tag.find_next().text)
        if date_str is None:
            date_str = re.search(date_regex_pattern1, tag.next.text)
        print(date_str)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            qtr_date_cleaned = qtr_date.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            date_str_cleaned = date_str.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            print(qtr_date_cleaned, date_str_cleaned)

            if qtr_date_cleaned == date_str_cleaned:
                html_table = tag.find_next('table')
                new_table = pd.read_html(
                    html_table.prettify(), skiprows=0, flavor='bs4')[0]
                new_table = new_table.applymap(lambda x: unicodedata.normalize(
                    'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                new_table = new_table.replace(
                    r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                new_table = new_table.dropna(how='all', axis=0)

                if master_table is None:
                    master_table = new_table
                else:
                    master_table = pd.concat(
                        [master_table, new_table], ignore_index=True)

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    return master_table

In [161]:
for index, (url, reporting_date) in enumerate(zip(filing_links['url'], filing_links['Reporting date'])):

    try:
        content = helper.get_content(
            helper.get_response(url=url, headers=headers))
    except Exception as e:
        print(f'failed to get the content: {e}')
    consolidated_schedule_regex = re.compile(
        r'(?i)^\s*.*\s*SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')

    for tag in content.find_all(text=consolidated_schedule_regex):
        print("Tag:", tag)
        # print("Find_next:", tag.find_next())
        date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
        print("Next_date:", re.search(date_regex_pattern1, tag.find_next().text))
        # print("next:", tag.next)
        # print("Next next: ", tag.find_next().next.next.next.text)
        # print("next sib: ", tag.find_next_sibling())
        print("\n")

/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_11149/3128941148.py:11: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for tag in content.find_all(text=consolidated_schedule_regex):


Tag: Schedule of Investments 
Next_date: None


Tag: Schedule of Investments 
Next_date: None


Tag: SCHEDULE OF INVESTMENTS (Unaudited)
Next_date: <re.Match object; span=(0, 18), match='September 30, 2023'>


Tag: SCHEDULE OF INVESTMENTS (Unaudited)
Next_date: <re.Match object; span=(0, 18), match='September 30, 2023'>


Tag: SCHEDULE OF INVESTMENTS (Unaudited)
Next_date: <re.Match object; span=(0, 18), match='September 30, 2023'>


Tag: SCHEDULE OF INVESTMENTS (Unaudited)
Next_date: <re.Match object; span=(0, 18), match='September 30, 2023'>


Tag: SCHEDULE OF INVESTMENTS (Unaudited)
Next_date: <re.Match object; span=(0, 18), match='September 30, 2023'>


Tag: SCHEDULE OF INVESTMENTS (Unaudited)
Next_date: <re.Match object; span=(0, 18), match='September 30, 2023'>


Tag: SCHEDULE OF INVESTMENTS (Unaudited)
Next_date: <re.Match object; span=(0, 18), match='September 30, 2023'>


Tag: SCHEDULE OF INVESTMENTS (Unaudited)
Next_date: <re.Match object; span=(0, 18), match='September 30, 2

for index, (url, reporting_date) in enumerate(zip(filing_links['url'], filing_links['Reporting date'])):

    try:
        content = helper.get_content(
            helper.get_response(url=url, headers=headers))
    except Exception as e:
        print(f'failed to get the content: {e}')

    master_table = extract_tables(content, reporting_date)
    master_table.to_csv(
        '../MT_csv_file/'+reporting_date.replace(',', '')+'.csv')

In [162]:
data_frames = []
data_frames_shapes = []


def extract_tables(soup_content, qtr_date):
    master_table = None
    print(qtr_date)

    consolidated_schedule_regex = re.compile(
        r'(?i)^\s*.*\s*SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    # date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):
        date_str = re.search(date_regex_pattern, tag.find_next().text)
        if date_str is None:
            date_str = re.search(date_regex_pattern, tag.next.text)
        if date_str is None:
            date_str = re.search(date_regex_pattern,
                                 tag.find_next().next.next.next.text)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            qtr_date_cleaned = qtr_date.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            date_str_cleaned = date_str.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            print(date_str, qtr_date_cleaned, date_str_cleaned)

            if qtr_date_cleaned == date_str_cleaned:
                html_table = tag.find_next('table')
                new_table = pd.read_html(
                    html_table.prettify(), keep_default_na=False, skiprows=0, flavor='bs4')[0]
                new_table = new_table.applymap(lambda x: unicodedata.normalize(
                    'NFKD', x.strip().strip(u'\u200b').replace('—', '0').replace('%', '')) if type(x) == str else x)

                new_table = new_table.replace(
                    r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                new_table = new_table.dropna(how='all', axis=0)

                if master_table is None:
                    master_table = new_table
                else:
                    master_table = pd.concat(
                        [master_table, new_table], ignore_index=True)
            # print(master_table)

            master_table = master_table.replace('N/A', 'No Value')

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    data_frames.append(master_table)
    data_frames_shapes.append(master_table.shape)
    return master_table

In [163]:
for index, (url, reporting_date) in enumerate(zip(filing_links['url'], filing_links['Reporting date'])):

    try:
        content = helper.get_content(
            helper.get_response(url=url, headers=headers))
    except Exception as e:
        print(f'failed to get the content: {e}')

    master_table = extract_tables(content, reporting_date)
    master_table.to_csv(
        '../MT_csv_file/'+reporting_date.replace(',', '')+'.csv')

September 30, 2023
September 30, 2023 september302023 september302023


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_11149/1484840400.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):


December 31, 2022 september302023 december312022
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september30202

/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_11149/1484840400.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):


December 31, 2022 june302023 december312022
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june302023 june302023
June 30, 2023 june30

...

September 30, 2017 september302017 september302017
September 30, 2017 september302017 september302017
September 30, 2017 september302017 september302017
September 30, 2017 september302017 september302017
September 30, 2017 september302017 september302017
September 30, 2017 september302017 september302017
September 30, 2017 september302017 september302017
September 30, 2017 september302017 september302017
September 30, 2017 september302017 september302017
September 30, 2017 september302017 september302017
September 30, 2017 september302017 september302017
September 30, 2017 september302017 september302017
September 30, 2017 september302017 september302017
September 30, 2017 september302017 september302017
September 30, 2017 september302017 september302017
March 31, 2017 september302017 march312017
March 31, 2017 september302017 march312017
March 31, 2017 september302017 march312017
March 31, 2017 september302017 march312017
March 31, 2017 september302017 march312017
March 31, 2017 septe

/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_11149/1484840400.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):


June 30, 2017 june302017 june302017
June 30, 2017 june302017 june302017
June 30, 2017 june302017 june302017
June 30, 2017 june302017 june302017
June 30, 2017 june302017 june302017
June 30, 2017 june302017 june302017
June 30, 2017 june302017 june302017
June 30, 2017 june302017 june302017
June 30, 2017 june302017 june302017
June 30, 2017 june302017 june302017
June 30, 2017 june302017 june302017
June 30, 2017 june302017 june302017
June 30, 2017 june302017 june302017
June 30, 2017 june302017 june302017
March 31, 2017 june302017 march312017
March 31, 2017 june302017 march312017
March 31, 2017 june302017 march312017
March 31, 2017 june302017 march312017
March 31, 2017 june302017 march312017
March 31, 2017 june302017 march312017
March 31, 2017 june302017 march312017
March 31, 2017 june302017 march312017
March 31, 2017 june302017 march312017
March 31, 2017 june302017 march312017
March 31, 2017 june302017 march312017
March 31, 2017 june302017 march312017
March 31, 2017 june302017 march312017
Ma

/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_11149/1484840400.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):


AttributeError: 'NoneType' object has no attribute 'replace'

In [169]:
helper

'https://www.sec.gov/Archives/edgar/data/0001278752/000127875217000047/ainv2018q110-q.htm'